### Recuperer "labo", "Equivalent traitement", "Annee commercialisation", "mois commercialisation", "prix","restriction age", "restriction poids" pour l'ibuprofène

In [ ]:
import requests
import pandas as pd
import urllib
import urllib.request
import re

labo = []
eqTraitements = []
AnneeCommer = []
moisCommer = []
prix = []
Age = []
Poids = []
def eqTraitement(inf):
    total = 0
    dosageSubstance = inf['compositions'][0]['substancesActives'][0]['dosageSubstance']
    designation = inf['compositions'][0]['designationElementPharmaceutique']
    nbComprimes = inf['presentations'][0]['libelle']
    nbComprimes = re.findall('[0-9]+',nbComprimes)
    dose = (int)(re.findall('[0-9]+',dosageSubstance)[0])
    unite = re.findall('[a-z]+',dosageSubstance)
    if designation == 'comprimé':
        total = dose * (int)(nbComprimes[0])
    elif designation == 'gel':
        referenceDosage = inf['compositions'][0]['referenceDosage']
        ref = (int)(re.findall('[0-9]+',referenceDosage)[0])
        a = (int)(nbComprimes[0])
        b = (int)(nbComprimes[1])
        total = a*b*dose/ref
    elif designation == 'solution':
        referenceDosage = inf['compositions'][0]['referenceDosage']
        ref = (int)(re.findall('[0-9]+',referenceDosage)[0])
        a = (int)(nbComprimes[0])
        b = (int)(nbComprimes[1])
        total = a*b*dose/ref
    else:
        print('ERROR')
    if 'mg' in unite:
        total = total/1000
    elif 'kg' in unite:
        total = total*1000
    return total


def getAllCIS(link):
    ibuprofenes = requests.get(link).json()
    codes = []
    for e in ibuprofenes:
        codes.append(e['codeCIS'])
    return codes

def getDataForACIS(code):
    code = str(code)
    url = 'https://www.open-medicaments.fr/api/v1/medicaments/'+code 
    inf = requests.get(url).json()
    texte = requests.get(url).text
    eqTraitements.append(str(eqTraitement(inf))+" grammes")
    date = inf['presentations'][0]['dateDeclarationCommercialisation']
    AnneeCommer.append(date[0:4])
    moisCommer.append(date[5:7])
    lab = inf['titulaires'][0]
    age = re.findall('([0-9]+) ans', texte)
    poids = re.findall('([0-9]+) kg', texte)
    ageRestriction = 'None'
    poidsRestriction = 'None'
    if len(age)>0:
        ageRestriction = age[0] + " ans"
    if len(poids)>0:
        poidsRestriction = poids[0] + " kgs"
    Age.append(ageRestriction)
    Poids.append(poidsRestriction)
    labo.append(lab)

def convertToDf():
    df=pd.DataFrame(labo,columns = ['LABO'])
    df["ANNEE"] = AnneeCommer
    df["MOIS"] = moisCommer
    df["EQ TRAITEMENT"] = eqTraitements
    df["AGE RESTRICTION"] = Age
    df["POIDS RESTRICTION"] = Poids
    return df

def mainMethod(medicament):
    medicament = str(medicament)
    url = 'https://www.open-medicaments.fr/api/v1/medicaments?query='+medicament
    codes = getAllCIS(url)
    for code in codes:
        getDataForACIS(code)
    df = convertToDf()
    return df

### Choisissez le médicament que vous souhaitez consulter

In [ ]:
mainMethod('ibuprofene')